# Demonstration notebook for the mailcom package
*Scientific Software Center, University of Heidelberg, December 2024*
The `mailcom` package is used to anonymize/pseudonymize textual data, i.e. email content. It takes an `eml` or `html` file as input and extracts information about attachements, number of attachements and type, and the content of the email body. The latter is then parsed through [`spaCy`](https://spacy.io/) and divided into sentences. The sentences are fed to a [`transformers`](https://huggingface.co/docs/transformers/en/index) named entity recognition (NER) [pipeline](https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/pipelines), and person names, places, organizations, miscellaneous, are detected in the inference task. Names are replaced by pseudos, while locations, organizations and miscellaneous are replaced by `[location]`, `[organization]` and `[misc]`. The text is further parsed using string methods, to replace any numbers with `[number]` and email addresses with `[email]`. The processed text and metadata can then be written to an `xml` file or into a pandas dataframe.

Please note that 100% accuracy is not possible with this task. Any output needs to be further checked by a human to ensure the text has been anonymized completely.

The current set-up is for Romance languages, however [other language models](https://spacy.io/usage/models) can also be loaded into the spaCy pipeline. The transformers pipeline uses the `xlm-roberta-large-finetuned-conll03-english` model revision number `18f95e9` by default, but other models can also be passed (see below).

Before using the `mailcom` package, please install it into your conda environment using
```
pip install mailcom
```
After that, select the appropriate kernel for your Jupyter notebook and execute the cell below to import the package. The package is currently under active development and any function calls are subject to changes.

In [ ]:
import mailcom.inout
import mailcom.parse
import pandas as pd
from IPython.display import display, HTML

The cell below defines a function used to display the result in the end, and highlight all named entities found in the text. It is used for demonstration purposes in this example.

Generally, this is a simple approach at highlighting the replaced pseudonyms, but the method itself is prone to errors and should only be used with care when assessing if the text has been anonymized correctly.

In [ ]:
# a dictionary matching colors to the different entity types
colors = {
    "LOC": "green",
    "ORG": "blue",
    "MISC": "yellow",
    "PER": "red"
}

# function for displaying the result using HTML
def highlight_ne(text, ne_list):
    # create a list of all entities with their positions
    entities = []
    for ne in ne_list:
        # avoid substituting the same entity multiple times
        if ne["word"] not in entities and ne["entity_group"] in colors:
            entities.append((ne, colors.get(ne["entity_group"])))
    
    # sort entities by their positions in the text in reverse order
    # is this necessary?
    # entities = sorted(entities, key=lambda x: x[0]["start"], reverse=True)

    # replace all "<" and ">" which may mess up spans
    text = text.replace("<", "&lt;")
    text = text.replace(">", "&gt;")
    # replace entities with highlighted spans
    for entity, color in entities:
        ent_word = entity["word"]
        # I think it may be overwriting the already replaced ones
        # Instead, maybe sort which ones are different and not a subset?
        text = text.replace(ent_word, f"<span style=\"background-color:{color}\">{ent_word}</span>")
    
    return text

Below, the input files are loaded from the given `input_dir` directory. You can provide relative or absolute paths to the directory that contains your `eml` or `html` files. All files of the `eml` or `html` file type in that directory will be considered input files.

In [ ]:
# import files from input_dir - change this to your own directory
input_dir = "../data/in/test"

io = mailcom.inout.InoutHandler(directory_name = input_dir)

# some internal processing
io.list_of_files()
# extracts the text of all emails in the directory and cleans up html content
io.process_emails()

# create pseudonymization object and load spacy and transformers
# set the spacy language for sentence splitting
spacy_language = "fr"
# you may also set the model using `model = "fr_core_news_md"`
spacy_model = "default"
# set the model for transformers, here using the default model
transformers_model = "xlm-roberta-large-finetuned-conll03-english"
# set the revision number for transformers, here using the default revision number
transformers_revision_number = "18f95e9"
ps = mailcom.parse.Pseudonymize()
ps.init_spacy(language=spacy_language, model=spacy_model)
ps.init_transformers(model=transformers_model, model_revision_number=transformers_revision_number)

In the cell below, the emails are looped over and the text is extracted. The text is then split into sentences and the sentences are pseudonymized. The pseudonymized sentences are then joined back into a text and saved to a new file.

The input text is displayed and the found named entities are highlighted for demonstration. Note that emails (all words containing '@') are filtered out seperately and thus not highlighted here.

In [ ]:
# loop over mails and pseudonymize them
for _, email in enumerate(io.get_email_list()):
    # the email text and metadata are stored in a dict
    # the dict already has the entries content, date, attachments, attachment type
    if not email["content"]:
        continue
    # Test functionality of Pseudonymize class
    # The output text is returned, as well as saved as a dict entry as "pseudo_content"
    _ = ps.pseudonymize(email)

    # display original text and highlight found and replaced NEs
    highlighted_html = highlight_ne(email["content"], ps.ne_list)
    display(HTML(highlighted_html))

After this, the output can be written to a file or processed further. The output is a list of dictionaries, each containing the metadata of the email and the pseudonymized content. In the below cell, the output is saved in a pandas dataframe.

In [ ]:
# write output to pandas df
df = pd.DataFrame(io.get_email_list())

The output can be saved as a csv file as well.

In [ ]:
io.write_csv("../data/out/out_demo.csv")